<a href="https://colab.research.google.com/github/ibribr/DT8807/blob/master/Stego_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pillow

In [1]:
from google.colab import files
from PIL import Image

uploaded = files.upload()
image_path = list(uploaded.keys())[0]  # Take the first uploaded file

print(image_path)
im = Image.open(image_path)
w, h = im.size
print('width:  ', w)
print('height: ', h)

Saving dog.png to dog.png
dog.png
width:   750
height:  579


In [2]:
def message_to_bin(message):
    binary_message = ''.join(format(ord(i), '08b') for i in message)
    return binary_message

def encode_image(image_path, secret_message):
    img = Image.open(image_path)
    encoded_img = img.copy()
    binary_secret_msg = message_to_bin(secret_message) + '1111111111111110'  # 16-bit delimiter which indicates end of the secret message
    print(len(binary_secret_msg))
    print(binary_secret_msg)

    data_index = 0
    for row in range(img.size[0]):
        for col in range(img.size[1]):
            pixel = list(img.getpixel((row, col)))
            for color in range(3):
                if data_index < len(binary_secret_msg):
                    pixel[color] = int(format(pixel[color], '08b')[:-1] + binary_secret_msg[data_index], 2)
                    data_index += 1
            encoded_img.putpixel((row, col), tuple(pixel))

    return encoded_img

if __name__ == '__main__':
    # Path to the image you want to hide the secret in
    #image_path = 'path/to/image.png'

    # Secret message you want to hide
    secret_message = "Hello, this is Ibrahim at UDST Qatar, today."

    encoded_image = encode_image(image_path, secret_message)
    encoded_image.save("encoded_image.png")


368
01001000011001010110110001101100011011110010110000100000011101000110100001101001011100110010000001101001011100110010000001001001011000100111001001100001011010000110100101101101001000000110000101110100001000000101010101000100010100110101010000100000010100010110000101110100011000010111001000101100001000000111010001101111011001000110000101111001001011101111111111111110


In [26]:
print(image_path)

dog.png


In [3]:
# decode the message from the encoded image
#from google.colab import files

#uploaded = files.upload()
encoded_image_path = '/content/encoded_image.png'#list(uploaded.keys())[0]  # Take the first uploaded file

def bin_to_message(binary_message):
    binary_values = [binary_message[i:i+8] for i in range(0, len(binary_message), 8)]
    ascii_characters = [chr(int(bv, 2)) for bv in binary_values]
    decoded_message = ''.join(ascii_characters)
    return decoded_message

def decode_image(encoded_image_path):
    img = Image.open(encoded_image_path)
    binary_secret_msg = ""

    for row in range(img.size[0]):
        for col in range(img.size[1]):
            pixel = list(img.getpixel((row, col)))
            for color in range(3):
                binary_secret_msg += (format(pixel[color], '08b')[-1])

    end_delimiter = '1111111111111110'
    end_delimiter_pos = binary_secret_msg.find(end_delimiter)
    if end_delimiter_pos != -1:
        binary_secret_msg = binary_secret_msg[:end_delimiter_pos]

    decoded_message = bin_to_message(binary_secret_msg)
    return decoded_message

secret_message = decode_image(encoded_image_path)
print(f"The hidden message is: {secret_message}")

The hidden message is: Hello, this is Ibrahim at UDST Qatar, today.
